# Notebook 1 – Data Cleaning, Feature Engineering, & Entity Resolution
**Project:** Judicial Vacancy → Nomination/Confirmation Pipeline

*Initial draft generated via ChatGPT model o3 on 2025-07-12T02:40:38.399372Z*

In [ ]:

import sys
from pathlib import Path

import pandas as pd
from loguru import logger
from rapidfuzz import fuzz, process

# Add the project root to the path so we can import our modules
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))


# Setup logging
logger.remove()  # Remove default handler
logger.add(sys.stderr, format="<green>{time:YYYY-MM-DD HH:mm:ss}</green> | <level>{level}</level> | <cyan>{function}</cyan> - <level>{message}</level>", level="INFO")

1

## Load dataframes from Raw data folder

Start with loading simpler, non-JSON-containing CSV files

In [ ]:
from nomination_predictor.config import INTERIM_DATA_DIR, RAW_DATA_DIR

# load FJC dataframes (and derived seat timeline)
fjc_judges = pd.read_csv(RAW_DATA_DIR / "judges.csv")
fjc_federal_judicial_service = pd.read_csv(RAW_DATA_DIR / "federal_judicial_service.csv")
fjc_demographics = pd.read_csv(RAW_DATA_DIR / "demographics.csv")
fjc_education = pd.read_csv(RAW_DATA_DIR / "education.csv")
fjc_other_federal_judicial_service = pd.read_csv(
    RAW_DATA_DIR / "other_federal_judicial_service.csv"
)
fjc_other_nominations_recess = pd.read_csv(RAW_DATA_DIR / "other_nominations_recess.csv")
seat_timeline = pd.read_csv(RAW_DATA_DIR / "seat_timeline.csv")

2025-07-14 00:50:51.423 | INFO     | nomination_predictor.config:<module>:103 - Project root: /home/wsl2ubuntuuser/nomination_predictor
2025-07-14 00:50:51.425 | INFO     | nomination_predictor.config:<module>:134 - Configuration loaded


Flatten JSON-containing congress DataFrames into separate DataFrames

In [ ]:
from nomination_predictor.features import flatten_json_dataframe

# Load Congress API dataframes
cong_nominations_raw = pd.read_csv(RAW_DATA_DIR / "nominations.csv")
cong_nominees_raw = pd.read_csv(RAW_DATA_DIR / "nominees.csv")

cong_nominations = flatten_json_dataframe(
    df=cong_nominations_raw,
    json_col="nomination",  # column containing the JSON data
    max_list_index=10,      # maximum number of list items to extract
    separator="_"           # separator for nested keys
)

cong_nominees= flatten_json_dataframe(
    df=cong_nominees_raw,
    json_col="nominee",
    max_list_index=5
)

2025-07-14 00:50:51.828 | INFO     | nomination_predictor.features:flatten_json_dataframe:282 - Flattening JSON data from column 'nomination' in 5746 rows
2025-07-14 00:50:54.309 | INFO     | nomination_predictor.features:flatten_json_dataframe:308 - Flattening complete. Original columns: 4, New columns: 37
2025-07-14 00:50:54.311 | INFO     | nomination_predictor.features:flatten_json_dataframe:282 - Flattening JSON data from column 'nominee' in 5671 rows
2025-07-14 00:50:55.676 | INFO     | nomination_predictor.features:flatten_json_dataframe:308 - Flattening complete. Original columns: 3, New columns: 34


In [ ]:
# Combine all dataframes into a single dictionary for bulk operations
# Start with FJC dataframes
dfs = {
    # FJC dataframes
    "fjc_judges": fjc_judges,
    "fjc_federal_judicial_service": fjc_federal_judicial_service,
    "fjc_demographics": fjc_demographics,
    "fjc_education": fjc_education,
    "fjc_other_federal_judicial_service": fjc_other_federal_judicial_service,
    "fjc_other_nominations_recess": fjc_other_nominations_recess,
    "seat_timeline": seat_timeline,
    
    # Congress dataframes
    "cong_nominations": cong_nominations,
    "cong_nominees": cong_nominees,
}

In [ ]:
# Print summary of available dataframes
print("Available dataframes:")
for name, df in dfs.items():
    print(f"- {name}: {len(df)} rows × {len(df.columns)} columns")

Available dataframes:
- fjc_judges: 4022 rows × 201 columns
- fjc_federal_judicial_service: 4720 rows × 30 columns
- fjc_demographics: 4022 rows × 18 columns
- fjc_education: 8040 rows × 6 columns
- fjc_other_federal_judicial_service: 611 rows × 31 columns
- fjc_other_nominations_recess: 828 rows × 4 columns
- seat_timeline: 4720 rows × 31 columns
- cong_nominations: 5746 rows × 37 columns
- cong_nominees: 5671 rows × 34 columns


Cong_nominee_orgs and cong_nominee_edu

JSON-containing files we can explode and/or flatten several different ways.  Whichever one is best depends on the use case.  Below is the method I settled on so far:

In [ ]:
# commented out because function this calls would throw warnings for what is by now known and tolerated table conditions

#from nomination_predictor.dataset import check_id_uniqueness
## Check each DataFrame for uniqueness of citation field
#print("Checking uniqueness of nomination/nominee identifiers...")
#for name, df in dfs.items():
#    if name.startswith("fjc_"):
#        logger.info(f"\n- Checking {name}...")
#        col="nid"
#        if col in df.columns:
#            check_id_uniqueness(df, id_field=col)
#        else:
#            logger.info(f"  Skipped: {col} column not found in {name}")
#    if name.startswith("cong_"):
#        logger.info(f"\n- Checking {name}...")
#        col="citation"
#        if col in df.columns:
#            check_id_uniqueness(df, id_field=col)
#        else:
#            logger.info(f"  Skipped: {col} column not found in {name}")

In [ ]:
# commented this cell out because IMO it's too early in this notebook to be worthwhile to save these as CSVs

## Save extracted tables to interim directory
#for name, df in dfs.items():
#    if len(df) > 0:  # Only save non-empty DataFrames
#        output_path = INTERIM_DATA_DIR / f"{name}.csv"
#        df.to_csv(output_path, index=False)
#        print(f"Saved {len(df)} records to {output_path}")

#### Quick peek at all loaded dataframes

In [ ]:
logger.info("Checking for general shape and first handfuls of rows")
for name, df in dfs.items():
    print(f"{name:<35} → {df.shape}")
    print(df.head())  

2025-07-14 00:50:55.729 | INFO     | __main__:<module>:1 - Checking for general shape and first handfuls of rows
fjc_judges                          → (4022, 201)
        nid       jid last_name first_name middle_name suffix  birth_month  \
0  13761857  13761857   Abelson       Adam         Ben    NaN          NaN   
1   1393931      3419    Abrams     Ronnie                             NaN   
2   1376976         1   Abruzzo    Matthew          T.                 4.0   
3  13651551  13651551     Abudu      Nancy       Gbana    NaN          NaN   
4   1376981         2   Acheson     Marcus      Wilson                 6.0   

   birth_day birth_year  birth_city  ... degree_(3)  degree_year_(3)  \
0        NaN       1982   Cleveland  ...        NaN              NaN   
1        NaN       1968    New York  ...        NaN              NaN   
2       30.0       1889    Brooklyn  ...        NaN              NaN   
3        NaN       1974  Alexandria  ...        NaN              NaN   
4       

In [ ]:
logger.info("Checking for null values")
    
for name, df in dfs.items():
    print(df.isnull().sum())

2025-07-14 00:50:55.785 | INFO     | __main__:<module>:1 - Checking for null values
nid                                         0
jid                                         0
last_name                                   0
first_name                                  0
middle_name                                35
                                         ... 
school_(5)                               4017
degree_(5)                               4018
degree_year_(5)                          4017
professional_career                         4
other_nominations/recess_appointments    3307
Length: 201, dtype: int64
nid                                     0
sequence                                0
judge_name                              0
court_type                              0
court_name                              0
appointment_title                       0
appointing_president                    0
party_of_appointing_president          39
reappointing_president               4710
party_

## Data cleaning

## Normalize column names for DataFrames

In [ ]:
print("=== Column Names Before ===")

for name, df in dfs.items():
    print(f"{name:<35} → {df.columns.tolist()}")

=== Column Names Before ===
fjc_judges                          → ['nid', 'jid', 'last_name', 'first_name', 'middle_name', 'suffix', 'birth_month', 'birth_day', 'birth_year', 'birth_city', 'birth_state', 'death_month', 'death_day', 'death_year', 'death_city', 'death_state', 'gender', 'race_or_ethnicity', 'court_type_(1)', 'court_name_(1)', 'appointment_title_(1)', 'appointing_president_(1)', 'party_of_appointing_president_(1)', 'reappointing_president_(1)', 'party_of_reappointing_president_(1)', 'aba_rating_(1)', 'seat_id_(1)', 'statute_authorizing_new_seat_(1)', 'recess_appointment_date_(1)', 'nomination_date_(1)', 'committee_referral_date_(1)', 'hearing_date_(1)', 'judiciary_committee_action_(1)', 'committee_action_date_(1)', 'senate_vote_type_(1)', 'ayes/nays_(1)', 'confirmation_date_(1)', 'commission_date_(1)', 'service_as_chief_judge,_begin_(1)', 'service_as_chief_judge,_end_(1)', '2nd_service_as_chief_judge,_begin_(1)', '2nd_service_as_chief_judge,_end_(1)', 'senior_status_date_(

In [ ]:
# call features.py's normalize_columns function on all DataFrames in dfs, and strip leading and trailing whitespace in all strings
from nomination_predictor.features import normalize_dataframe_columns

for name, df in dfs.items():
    df = normalize_dataframe_columns(df)
    df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
    dfs[name] = df

In [ ]:
print("=== Column Names After ===")

for name, df in dfs.items():
    print(f"{name:<35} → {df.columns.tolist()}")

=== Column Names After ===
fjc_judges                          → ['nid', 'jid', 'last_name', 'first_name', 'middle_name', 'suffix', 'birth_month', 'birth_day', 'birth_year', 'birth_city', 'birth_state', 'death_month', 'death_day', 'death_year', 'death_city', 'death_state', 'gender', 'race_or_ethnicity', 'court_type_(1)', 'court_name_(1)', 'appointment_title_(1)', 'appointing_president_(1)', 'party_of_appointing_president_(1)', 'reappointing_president_(1)', 'party_of_reappointing_president_(1)', 'aba_rating_(1)', 'seat_id_(1)', 'statute_authorizing_new_seat_(1)', 'recess_appointment_date_(1)', 'nomination_date_(1)', 'committee_referral_date_(1)', 'hearing_date_(1)', 'judiciary_committee_action_(1)', 'committee_action_date_(1)', 'senate_vote_type_(1)', 'ayes/nays_(1)', 'confirmation_date_(1)', 'commission_date_(1)', 'service_as_chief_judge,_begin_(1)', 'service_as_chief_judge,_end_(1)', '2nd_service_as_chief_judge,_begin_(1)', '2nd_service_as_chief_judge,_end_(1)', 'senior_status_date_(1

Left-merge nominees table onto nominations table

In [ ]:
from nomination_predictor.features import merge_nominees_onto_nominations

try:
    # Assuming cong_nominations and cong_nominees dataframes are already loaded
    cong_noms = merge_nominees_onto_nominations(dfs["cong_nominations"], dfs["cong_nominees"])
    
    # Show sample of the merged dataframe
    display(cong_noms.head())
    
    # Report on the merge results
    logger.info(f"Original nominations shape: {cong_nominations.shape}")
    logger.info(f"Original nominees shape: {cong_nominees.shape}")
    logger.info(f"Merged dataframe shape: {cong_noms.shape}")
    
    dfs["cong_noms"] = cong_noms
    
except NameError:
    logger.error("Required dataframes (cong_nominations, cong_nominees) are not defined")
except Exception as e:
    logger.error(f"Error in merge process: {e}")

2025-07-14 00:50:56.365 | INFO     | nomination_predictor.features:merge_nominees_onto_nominations:551 - Extracted 5671 URLs from nominees request column (100.0% of rows)
2025-07-14 00:50:56.367 | INFO     | nomination_predictor.features:merge_nominees_onto_nominations:575 - Nominations dataframe has 5667 non-null URLs (98.6% of rows)
2025-07-14 00:50:56.402 | INFO     | nomination_predictor.features:merge_nominees_onto_nominations:591 - Merged dataframe has 6150 rows
2025-07-14 00:50:56.402 | INFO     | nomination_predictor.features:merge_nominees_onto_nominations:592 - Successfully matched 6071 nominations with nominees (98.7%)


request retrieval_date  \
0  {'congress': '118', 'contentType': 'applicatio...     2025-07-12   
1  {'congress': '118', 'contentType': 'applicatio...     2025-07-12   
2  {'congress': '118', 'contentType': 'applicatio...     2025-07-12   
3  {'congress': '118', 'contentType': 'applicatio...     2025-07-12   
4  {'congress': '118', 'contentType': 'applicatio...     2025-07-12   

   is_full_detail  actions_count  \
0            True            6.0   
1            True            6.0   
2            True           11.0   
3            True           14.0   
4            True           20.0   

                                         actions_url authoritydate citation  \
0  https://api.congress.gov/v3/nomination/118/201...    2025-05-12   PN2013   
1  https://api.congress.gov/v3/nomination/118/201...    2025-05-12   PN2012   
2  https://api.congress.gov/v3/nomination/118/813...    2025-03-28    PN813   
3  https://api.congress.gov/v3/nomination/118/903...    2025-03-28    PN903   
4  https://api.congress.gov/v3/nomination/118/816...    2025-03-28    PN816   

   committees_count                                     committees_url  \
0               1.0  https://api.congress.gov/v3/nomination/118/201...   
1               1.0  https://api.congress.gov/v3/nomination/118/201...   
2               1.0  https://api.congress.gov/v3/nomination/118/813...   
3               1.0  https://api.congress.gov/v3/nomination/118/903...   
4               1.0  https://api.congress.gov/v3/nomination/118/816...   

   congress  ... nominees_3_ordinal  nominees_3_state nominees_4_firstname  \
0       118  ...                NaN              None                 None   
1       118  ...                NaN              None                 None   
2       118  ...                NaN              None                 None   
3       118  ...                NaN              None                 None   
4       118  ...                NaN              None                 None   

  nominees_4_lastname nominees_4_ordinal  nominees_4_state  nominees_3_suffix  \
0                None                NaN              None               None   
1                None                NaN              None               None   
2                None                NaN              None               None   
3                None                NaN              None               None   
4                None                NaN              None               None   

   nominees_1_middlename nominees_3_middlename nominees_4_middlename  
0                   None                  None                  None  
1                   None                  None                  None  
2                   None                  None                  None  
3                   None                  None                  None  
4                   None                  None                  None  

[5 rows x 70 columns]

2025-07-14 00:50:56.419 | INFO     | __main__:<module>:11 - Original nominations shape: (5746, 37)
2025-07-14 00:50:56.420 | INFO     | __main__:<module>:12 - Original nominees shape: (5671, 34)
2025-07-14 00:50:56.420 | INFO     | __main__:<module>:13 - Merged dataframe shape: (6150, 70)


### Drop non-judge nominations based on position title

In [ ]:
# Filter out non-judicial nominations using the function from features.py
from nomination_predictor.features import filter_non_judicial_nominations

# Define non-judicial titles to filter out
non_judicial_titles = [
    "Attorney", "Board", "Commission", "Director", "Marshal",
    "Assistant", "Representative", "Secretary of", "Member of"
]

dfs["cong_noms"] = filter_non_judicial_nominations(dfs["cong_noms"], non_judicial_titles=non_judicial_titles)
cong_noms = dfs["cong_noms"]

2025-07-14 00:50:56.467 | INFO     | nomination_predictor.features:filter_non_judicial_nominations:185 - Found 1331 unique citations with non-judicial titles
2025-07-14 00:50:56.474 | INFO     | nomination_predictor.features:filter_non_judicial_nominations:191 - Removed 4393/6150 corresponding records


### Convert date strings to datetime objects

In [ ]:
# for any columns which contain certain keywords in their column name and contain string values, convert from string to datetime
datetime_related_keywords = ("date", "year", "month")

for name, df in dfs.items():
    for col in df.columns:
        if any(keyword in col for keyword in datetime_related_keywords) and df[col].dtype == "object":
            logger.info(f"Converting {col} to datetime for {name}")
            df[col] = pd.to_datetime(df[col], errors="coerce")

2025-07-14 00:50:56.486 | INFO     | __main__:<module>:7 - Converting birth_year to datetime for fjc_judges
2025-07-14 00:50:56.493 | INFO     | __main__:<module>:7 - Converting recess_appointment_date_(1) to datetime for fjc_judges
2025-07-14 00:50:56.497 | INFO     | __main__:<module>:7 - Converting nomination_date_(1) to datetime for fjc_judges
2025-07-14 00:50:56.499 | INFO     | __main__:<module>:7 - Converting committee_referral_date_(1) to datetime for fjc_judges
2025-07-14 00:50:56.502 | INFO     | __main__:<module>:7 - Converting hearing_date_(1) to datetime for fjc_judges
2025-07-14 00:50:56.506 | INFO     | __main__:<module>:7 - Converting committee_action_date_(1) to datetime for fjc_judges
2025-07-14 00:50:56.510 | INFO     | __main__:<module>:7 - Converting confirmation_date_(1) to datetime for fjc_judges
2025-07-14 00:50:56.514 | INFO     | __main__:<module>:7 - Converting commission_date_(1) to datetime for fjc_judges
2025-07-14 00:50:56.516 | INFO     | __main__:<modul

### Normalize all string values we'll later have to fuzzy-match

In [ ]:
keywords_which_denote_string_columns_to_normalize = ("court", "circuit", "district", "description", "name")

for name, df in dfs.items():
    for col in df.columns:
        if any(keyword in col.casefold() for keyword in keywords_which_denote_string_columns_to_normalize) and df[col].dtype == object:
            logger.info(F"Normalizing all values within column named {col} in {name}")
            df[col] = df[col].str.casefold()

2025-07-14 00:50:56.753 | INFO     | __main__:<module>:6 - Normalizing all values within column named last_name in fjc_judges
2025-07-14 00:50:56.756 | INFO     | __main__:<module>:6 - Normalizing all values within column named first_name in fjc_judges
2025-07-14 00:50:56.758 | INFO     | __main__:<module>:6 - Normalizing all values within column named middle_name in fjc_judges
2025-07-14 00:50:56.759 | INFO     | __main__:<module>:6 - Normalizing all values within column named court_type_(1) in fjc_judges
2025-07-14 00:50:56.762 | INFO     | __main__:<module>:6 - Normalizing all values within column named court_name_(1) in fjc_judges
2025-07-14 00:50:56.764 | INFO     | __main__:<module>:6 - Normalizing all values within column named court_type_(2) in fjc_judges
2025-07-14 00:50:56.766 | INFO     | __main__:<module>:6 - Normalizing all values within column named court_name_(2) in fjc_judges
2025-07-14 00:50:56.767 | INFO     | __main__:<module>:6 - Normalizing all values within column

### Count and display unique values under each column

In [ ]:
# display counts of unique values in DataFrame columns:
for name, df in dfs.items():
    for col in sorted(df.columns):
     print(f"{name} - {col}: {df[col].nunique()} unique values")

fjc_judges - 2nd_service_as_chief_judge,_begin_(1): 5 unique values
fjc_judges - 2nd_service_as_chief_judge,_begin_(2): 0 unique values
fjc_judges - 2nd_service_as_chief_judge,_begin_(3): 0 unique values
fjc_judges - 2nd_service_as_chief_judge,_begin_(4): 0 unique values
fjc_judges - 2nd_service_as_chief_judge,_begin_(5): 0 unique values
fjc_judges - 2nd_service_as_chief_judge,_begin_(6): 0 unique values
fjc_judges - 2nd_service_as_chief_judge,_end_(1): 4 unique values
fjc_judges - 2nd_service_as_chief_judge,_end_(2): 0 unique values
fjc_judges - 2nd_service_as_chief_judge,_end_(3): 0 unique values
fjc_judges - 2nd_service_as_chief_judge,_end_(4): 0 unique values
fjc_judges - 2nd_service_as_chief_judge,_end_(5): 0 unique values
fjc_judges - 2nd_service_as_chief_judge,_end_(6): 0 unique values
fjc_judges - aba_rating_(1): 6 unique values
fjc_judges - aba_rating_(2): 5 unique values
fjc_judges - aba_rating_(3): 3 unique values
fjc_judges - aba_rating_(4): 1 unique values
fjc_judges - aba

### Set nid as index (for the couple of FJC dataframes designed to use 'nid' uniquely)

In [ ]:
# For the dataframes that have unique nid, set them as the index to optimize lookups/joins
dfs["fjc_judges"].set_index('nid', drop=False, inplace=True, verify_integrity=True)
dfs["fjc_demographics"].set_index('nid', drop=False, inplace=True, verify_integrity=True)

## Fuzzy-matching FJC judges to Congress.gov nominees

### Preparing columns to aid matching

In [ ]:
# add a "full_name_concatenated" column to the fjc_federal_judicial_service dataframe which is composed by flipping its judge_name column values 
# from "lastname, firstname middleNameOrMiddleInitial (, optional comma and suffix)" to "firstname lastname middle suffix"
from nomination_predictor.features import \
    convert_judge_name_format_from_last_comma_first_to_first_then_last

try:
    dfs["fjc_federal_judicial_service"]["full_name_concatenated"] = dfs["fjc_federal_judicial_service"]["judge_name"].apply(convert_judge_name_format_from_last_comma_first_to_first_then_last)
    
    # Show some examples to verify the conversion
    sample = dfs["fjc_federal_judicial_service"][['judge_name', 'full_name_concatenated']].head(10)
    display(sample)
    
    # Count null values to check for any conversion failures
    null_count = dfs["fjc_federal_judicial_service"]["full_name_concatenated"].isna().sum()
    empty_count = (dfs["fjc_federal_judicial_service"]["full_name_concatenated"] == '').sum()
    
    if null_count > 0 or empty_count > 0:
        print(f"Warning: Found {null_count} null values and {empty_count} empty strings in the converted names.")
        
    print(f"Successfully added 'full_name_concatenated' column to fjc_federal_judicial_service dataframe with {len(dfs["fjc_federal_judicial_service"])}) entries.")
    
except Exception as e:
    logger.error(f"Error creating full_name_concatenated column: {e}")
    # If there's an error, display the first few rows of fjc_federal_judicial_service to help diagnose
    logger.info("\nSample of fjc_federal_judicial_service dataframe:")
    display(dfs["fjc_federal_judicial_service"].head(3))
    logger.info(f"Columns available: {dfs["fjc_federal_judicial_service"].columns.tolist()}")

judge_name   full_name_concatenated
0          abelson, adam ben         adam ben abelson
1             abrams, ronnie            ronnie abrams
2        abruzzo, matthew t.       matthew t. abruzzo
3         abudu, nancy gbana        nancy gbana abudu
4     acheson, marcus wilson    marcus wilson acheson
5     acheson, marcus wilson    marcus wilson acheson
6     acheson, marcus wilson    marcus wilson acheson
7  acker, william marsh, jr.  william marsh acker jr.
8    ackerman, harold arnold   harold arnold ackerman
9      ackerman, james waldo     james waldo ackerman

Successfully added 'full_name_concatenated' column to fjc_federal_judicial_service dataframe with 4720) entries.


In [ ]:
# add a "full_name_from_description" and a "location_of_origin_from_description" columns to the dfs["cong_noms"] dataframe which regex-captures the first segments of the same dfs["cong_noms"] dataframe row's "description" string, 
# i.e. captures name before the first appearances of the phrases ", of " or ", of the "
# and captures location from the second segment of the same dfs["cong_noms"] dataframe row's "description" string
# i.e. captures between the above-seen phrase ", of " or ", of the " through to the phrase ", to be "
# examples: 
# melissa damian, of florida, to be ...  gets captured into those new columns as "melissa damian" and "florida"
# nicole g. bernerr of maryland, to be united... gets captured into those new columns as "nicole g. bernerr" and "maryland"
# kirk edward sherriff, of california, to be united... gets captured into those new columns as "kirk edward sherriff" and "california"
# sherri malloy beatty-arthur, of the district of columbia, for... gets captured into those new columns as "sherri malloy beatty-arthur" and "district of columbia"

# Extract full_name_from_description and location_of_origin_from_description from description field
from nomination_predictor.features import extract_name_and_location_columns

# Apply the extraction function to cong_noms dataframe
if 'cong_noms' in dfs:
    dfs['cong_noms'] = extract_name_and_location_columns(dfs['cong_noms'])
    
    # Display sample results to verify extraction
    sample_cols = ['description', 'full_name_from_description', 'location_of_origin_from_description']
    display(dfs['cong_noms'][sample_cols].head(10))
    
    # Report extraction statistics
    total_rows = len(dfs['cong_noms'])
    name_filled = dfs['cong_noms']['full_name_from_description'].notna().sum()
    location_filled = dfs['cong_noms']['location_of_origin_from_description'].notna().sum()
    
    print(f"Extracted names for {name_filled}/{total_rows} records ({name_filled/total_rows:.1%})")
    print(f"Extracted locations for {location_filled}/{total_rows} records ({location_filled/total_rows:.1%})")
else:
    print("Error: 'cong_noms' dataframe not found in dfs dictionary.")

2025-07-14 00:50:57.288 | INFO     | nomination_predictor.features:extract_name_and_location_columns:720 - Extracted 1757/1757 (100.0%) names and 1757/1757 (100.0%) locations


description  \
0   nicholas george miranda, of the district of co...   
1   james graham lake, of the district of columbia...   
16  mustafa taher kasubhai, of oregon, to be unite...   
17  mustafa taher kasubhai, of oregon, to be unite...   
21  jacqueline becerra, of florida, to be united s...   
22  mustafa taher kasubhai, of oregon, to be unite...   
23  mustafa taher kasubhai, of oregon, to be unite...   
24  jacquelyn d. austin, of south carolina, to be ...   
31  melissa damian, of florida, to be united state...   
36  nicole g. berner, of maryland, to be united st...   

   full_name_from_description location_of_origin_from_description  
0     nicholas george miranda                district of columbia  
1           james graham lake                district of columbia  
16     mustafa taher kasubhai                              oregon  
17     mustafa taher kasubhai                              oregon  
21         jacqueline becerra                             florida  
22     mustafa taher kasubhai                              oregon  
23     mustafa taher kasubhai                              oregon  
24        jacquelyn d. austin                      south carolina  
31             melissa damian                             florida  
36           nicole g. berner                            maryland

Extracted names for 1757/1757 records (100.0%)
Extracted locations for 1757/1757 records (100.0%)


In [ ]:
# Add a "last_name_from_full_name" column to dfs["cong_noms"] with only the last name

from nomination_predictor.features import extract_last_name

# Add the last_name_from_full_name column
dfs['cong_noms']['last_name_from_full_name'] = dfs['cong_noms']['full_name_from_description'].apply(extract_last_name)

# Display sample results to verify extraction
sample_cols = ['full_name_from_description', 'last_name_from_full_name']
display(dfs['cong_noms'][sample_cols].head(10))

# Count non-null values
last_name_count = dfs['cong_noms']['last_name_from_full_name'].notna().sum()
print(f"Extracted last names for {last_name_count}/{len(dfs['cong_noms'])} records ({last_name_count/len(dfs['cong_noms']):.1%})")

full_name_from_description last_name_from_full_name
0     nicholas george miranda                  miranda
1           james graham lake                     lake
16     mustafa taher kasubhai                 kasubhai
17     mustafa taher kasubhai                 kasubhai
21         jacqueline becerra                  becerra
22     mustafa taher kasubhai                 kasubhai
23     mustafa taher kasubhai                 kasubhai
24        jacquelyn d. austin                   austin
31             melissa damian                   damian
36           nicole g. berner                   berner

Extracted last names for 1757/1757 records (100.0%)


In [ ]:
# add a column "last_name" derived from judge_name to the dfs["fjc_federal_judicial_service"] dataframe because fuzzy-matcher function will look for that as the name of a blocking column later
from nomination_predictor.features import extract_last_name

if 'fjc_federal_judicial_service' in dfs:
    dfs['fjc_federal_judicial_service']['last_name'] = dfs['fjc_federal_judicial_service']['judge_name'].apply(extract_last_name)
    display(dfs['fjc_federal_judicial_service'][['judge_name', 'last_name']].head(10))
else:
    print("Error: 'fjc_federal_judicial_service' dataframe not found in dfs dictionary.")

judge_name last_name
0          abelson, adam ben   abelson
1             abrams, ronnie    abrams
2        abruzzo, matthew t.   abruzzo
3         abudu, nancy gbana     abudu
4     acheson, marcus wilson   acheson
5     acheson, marcus wilson   acheson
6     acheson, marcus wilson   acheson
7  acker, william marsh, jr.     acker
8    ackerman, harold arnold  ackerman
9      ackerman, james waldo  ackerman

In [ ]:
# perform date matching of fjc's nomination_date vs. congress' received_date +/- some threshold, e.g. 45 days

In [ ]:
# add a court_name_from_description column to the dfs["cong_noms"] dataframe which regex-captures the second segment of the same dataframe row's "description" string, 
# i.e. anything in between the phrase  ", to be " (the ones which used as the end-signifier of the "location" above) through to the first appearance of ", vice"

#### Blocking-based fuzzy matching

In [ ]:
# Import the entity matching module
from nomination_predictor.entity_matching import (
    generate_matching_summary, update_dataframe_with_matches)

# Set up the matching parameters
MATCH_THRESHOLD = 80  # Minimum score to consider a high-confidence match

In [ ]:
# utilize the capabilities provided by fuzzy_matching.py to identify which rows in dfs["fjc_federal_judicial_service"] correspond most closely with rows in dfs["cong_noms"].
# for any rows where a high-confidence-enough unambiguous match is found, copy the 'nid" from that row in dfs["fjc_federal_judicial_service"] to the corresponding row in dfs["cong_noms"]
# for any dfs["fjc_federal_judicial_service"] rows where only one possible row in dfs["cong_noms"] appears to be a possible match, but the match confidence is lower than our cutoff threshold, present those to the user.
# for any dfs["fjc_federal_judicial_service"] rows where matches appear ambiguous about which of multiple rows in dfs["cong_noms"] it could correlate to, present those to the user.
# for any dfs["fjc_federal_judicial_service"] rows where no matches are found, separately present those to the user.
# for any dfs["cong_noms"] rows where no matches are found, separately present those to the user.

In [ ]:
from nameparser import HumanName

# next analysis to try: name matching only.  
# Use nameparser.  
# Take dfs['cong_noms']['full_name_from_description'] column and dfs['fjc_federal_judicial_service']['judge_name'] column.
# compare last name only, using exact string matching, after a plain casefold() and whitespace-stripping.
# if exact match, show me both, and show me the fjc dataframe's 'nid' for that person.
# if multiple matches, show a logger.info() listing the multiple matches.
# then for anyone with multiple matches, compare first name (obtained via nameparser) only.
# if exact match, show me both, and show me the fjc dataframe's 'nid' for that person.
# if multiple matches, show a logger.info() listing the multiple matches.

In [ ]:
from nomination_predictor.name_matching import perform_exact_name_matching

results_of_name_matching_to_bridge_nids_to_congress_dataframe_indices= perform_exact_name_matching(
    congress_df=dfs["cong_noms"],
    fjc_df=dfs["fjc_federal_judicial_service"],
    congress_name_col="full_name_from_description",
    fjc_name_col="judge_name"
)

2025-07-14 00:59:08.061 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:47 - Starting exact name matching with 1757 Congress records and 4720 FJC records
2025-07-14 00:59:08.316 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:66 - Performing first-pass join on last and first name
2025-07-14 00:59:08.338 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:83 - Found 25 ambiguous matches, attempting middle initial disambiguation
2025-07-14 00:59:08.344 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:110 - Creating final results dataframe
2025-07-14 00:59:08.346 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:152 - Name matching complete: 976 total matches
2025-07-14 00:59:08.347 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:153 -   - 976 unambiguous matches
2025-07-14 00:59:08.347 | INFO     | nomination_predictor.name_match

In [ ]:

# Show results
results_of_name_matching_to_bridge_nids_to_congress_dataframe_indices.head()

congress_index           congress_name                 fjc_name  \
0              16  mustafa taher kasubhai  kasubhai, mustafa taher   
1              17  mustafa taher kasubhai  kasubhai, mustafa taher   
2              21      jacqueline becerra      becerra, jacqueline   
3              22  mustafa taher kasubhai  kasubhai, mustafa taher   
4              23  mustafa taher kasubhai  kasubhai, mustafa taher   

          nid           match_type  ambiguous  
0  13761892.0  first_and_last_name      False  
1  13761892.0  first_and_last_name      False  
2  13761516.0  first_and_last_name      False  
3  13761892.0  first_and_last_name      False  
4  13761892.0  first_and_last_name      False

In [ ]:
# keep only *unambiguous* pairs
nid_map = (
    results_of_name_matching_to_bridge_nids_to_congress_dataframe_indices[~results_of_name_matching_to_bridge_nids_to_congress_dataframe_indices["ambiguous"]]        # drop rows still ambiguous
      .set_index("congress_index")["nid"]
)

In [ ]:
# at long last, we have a way to bridge the gap between the congress.gov data and the fjc data

# we can now use the nid_map to add the nid column to the congress.gov data
dfs["cong_noms"]["nid"] = dfs["cong_noms"].index.to_series().map(nid_map)
cong_noms = dfs["cong_noms"]

In [ ]:
## FIXME: decide whether to save as separate vs. overwrite in interim folder
## Save the cleaned interim datasets for downstream notebooks
#cong_nominees.to_csv(INTERIM_DATA_DIR / "congress_nominees_cleaned.csv", index=False)
#fjc_judges.to_csv(INTERIM_DATA_DIR / "fjc_judges_cleaned.csv", index=False)
#fjc_service.to_csv(INTERIM_DATA_DIR / "fjc_service_cleaned.csv", index=False)

## Combining FJC data

### Handling nominees' education and job history

Before we combine FJC data, we have to consider whether/how to handle judges' education, job history, age, ABA rating, etc., because the only other table in the FJC data which handles nid uniquely is "demographics," which are unchanging.
The simplest way to handle the non-unique-nid tables it would be to left-merge on "nid" and only take the most recently-dated row.  In most cases this would likely land on keeping the most prestigious degree or job.

However, it is entirely likely a judge's education or job history has changed substantially since their first nomination, and affected their qualifications for each later nomination.

All of these indicate to me that it's worth considering the judge's position, education, etc., not as of the most recent records available, but instead _as of when they were nominated._

That means we can't do a simple left-join of all of our FJC data.  Instead, we have to -- using a combination of names, court locations, and vacancy dates -- fuzzy-match to find which "nid" corresponds to each "citation" in the Congress data, as our way of bridging between FJC judges and congress' nominee data. Then use the "received date" for that citation as a cutoff date for when we lookup education and job records by "nid" -- so we can avoid mistakenly linking to a citation any employemnt & job records dated after that cutoff date.

Thankfully we do have the school, degree, and degree_year in the education record, for both their bachelors and their masters and their associate degree(s) and LLB and J.D. etc., so we can look that up.  The education dataframe even comes with a "sequence" number for each education record, which is an even easier-to-use indicator of chronological order than the degree_year for any given "nid" lookup for a judge.

Job history is more challenging to deal with because literally every row entry in that dataframe lists it uniquely, but we do have the data available.  On early attempts, it may be simplest to ignore it; then feature-engineer basic booleans for whether they did/didn't have experience in common-phrase-identifiable positions such as "Private practice" or "Attorney general" or "Navy" or "Army" etc.; eventually a parser can look for the year spreads listed there as a rough indicator of amounts of experience gleaned from each professional role.

### Build predecessor lookup table

In [ ]:
# Create the predecessor lookup table
predecessor_lookup = get_predecessor_info(seat_timeline_df)
print(f"Created predecessor lookup: {len(predecessor_lookup)} records")

# Preview the predecessor lookup
print(predecessor_lookup.head())
all_dataframes['predecessor_lookup'] = predecessor_lookup